# Behind the pipeline

In [ ]:
from transformers import pipeline

## Preprocessing with a tokenizer

In [ ]:
# Pipeline groups together three steps: preprocessing, passing the inputs through the model, and postprocessing
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

In [ ]:
# Preprocessing with a tokenizer
# All preprocessing needs to be done in exactly the same way as when the model was pretrained
# To do this, we use the AutoTokenizer class and its from_pretrained() method.
# Using the checkpoint name of our model, it will automatically fetch the data associated with the model’s tokenizer and cache it
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt") #return_tensors: To specify the type of tensors we want to get back 
# truncation=True, Any sentence longer than the maximum the model can handle is truncated
print(inputs)

## Going through the model

In [ ]:
#cache body here
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint) # downloaded same checkpoint used in pipeline(cached already) and instantiated a model with it
model

### A high-dimensional vector?

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)
# torch.Size([2, 16, 768]) [Batch size, Sequence length, hidden size]

### Model heads: Making sense out of numbers

In [ ]:
from transformers import TFAutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
outputs = model(**inputs)
print(outputs.logits.shape) # torch.Size([2, 2])

## Postprocessing the output

In [ ]:
print(outputs.logits)

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

In [ ]:
model.config.id2label
#{0: 'NEGATIVE', 1: 'POSITIVE'}